In [44]:
from dotenv import load_dotenv
import os
from espn_api.football import League
import json

In [45]:
load_dotenv()

external_league_id = os.getenv('LEAGUE_ID')
swid = os.getenv('SWID')
espn_s2 = os.getenv('ESPN_S2')
year = 2024

In [46]:
league = League(league_id=external_league_id, year=year, espn_s2=espn_s2, swid=swid)

In [47]:
import os
from supabase import create_client, Client

def get_supabase_client():
    # load supabase client    
    supabase: Client = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_KEY"))
    return supabase

# supabase helper functions
def get_league_id_from_espn_league_id(supabase_client, external_league_id):
    response = supabase_client.table("leagues").select("id").eq("external_league_id", external_league_id).execute()
    return response.data[0]["id"]


def get_teams_in_league(supabase_client, league_id):
    response = supabase_client.table("teams").select("id, team_name, espn_team_id").eq("league_id", league_id).execute()
    return response.data

def get_espn_player_id_to_player_id_map(supabase_client):
    response = supabase_client.table("nfl_players").select("id, espn_player_id").execute()
    player_map = {}
    for player in response.data:
        player_map[player["espn_player_id"]] = player["id"]
    return player_map

In [52]:

def insert_league_draft_picks(league):
    if league.draft is None or len(league.draft) == 0:
        return

    external_league_id = league.league_id

    supabase_client = get_supabase_client()
    league_id = get_league_id_from_espn_league_id(supabase_client, external_league_id)
    teams_in_league = get_teams_in_league(supabase_client, league_id)
    espn_player_id_to_player_id_map = get_espn_player_id_to_player_id_map(supabase_client)
    
    draft_picks = []

    for draft_pick in league.draft:
        try:
            espn_team_id = draft_pick.team.team_id
            team_id = next(team["id"] for team in teams_in_league if team["espn_team_id"] == espn_team_id)
            player_id = espn_player_id_to_player_id_map[draft_pick.playerId]
            draft_picks.append({
                "league_id": league_id,
                "team_id": team_id,
                "player_id": player_id,
                "round": draft_pick.round_num,
                "pick": draft_pick.round_pick
            })
        except:
            draft_picks.append({
                "league_id": league_id,
                "team_id": None,
                "player_id": None,
                "round": draft_pick.round_num,
                "pick": draft_pick.round_pick
            })

    supabase_client.table("draft_picks").insert(draft_picks).execute() 
    

insert_league_draft_picks(league)

APIError: {'message': "Could not find the table 'public.draft_picks' in the schema cache", 'code': 'PGRST205', 'hint': "Perhaps you meant the table 'public.draft_order'", 'details': None}